In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pydicom
from tqdm import tqdm
import os
import numpy as np
import cv2

In [ ]:
# Install W&B 
!pip install -q --upgrade wandb
# Login 
import wandb
wandb.login()

FOR MALE FEMALE PA

In [ ]:
male_pa_df = pd.read_csv('../input/male-adult-pa-pneumonea/male_adult_pa.csv')
female_pa_df = pd.read_csv("../input/female-adult-pa-pneumonea/female_pa_adult.csv")
male_pa_df['gender'] = 0
female_pa_df['gender'] = 1
mf_df = pd.concat([male_pa_df,female_pa_df], ignore_index = True)
working_df = mf_df.copy()
working_df = working_df.drop(working_df[working_df['class'] == 'No Lung Opacity / Not Normal'].index)
working_df['new_class'] = working_df['class'].map({ 'Normal':'Non-Pneumonia','Lung Opacity': 'Pneumonia' })

working_df.rename(columns = {'Unnamed: 0' :'unique_id'}, inplace = True)
working_df.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
class_enc = LabelEncoder()
working_df['class_idx'] = class_enc.fit_transform(working_df['new_class'])
oh_enc = OneHotEncoder(sparse=False)
working_df['class_vec'] = oh_enc.fit_transform(
    working_df['class_idx'].values.reshape(-1, 1)).tolist() 

In [ ]:
from sklearn.model_selection import train_test_split
image_df = working_df.groupby('patientId').apply(lambda x: x.sample(1))
raw_train_df, remain_df = train_test_split(image_df, test_size=0.20, random_state=2018,
                                    stratify=image_df['new_class'])
#valid_df, test_df = train_test_split(remain_df, test_size=0.33, random_state=2018,
 #                                   stratify=remain_df['new_class'])
print(raw_train_df.shape, 'training data')
print(remain_df.shape, 'remain data')
#print(valid_df.shape, 'valid data')
#print(test_df.shape, 'test data')

In [ ]:

os.getcwd()

In [ ]:
!mkdir Train_Valid_images #in kaggle/working

In [ ]:
!mkdir Test_images

In [ ]:
!mkdir Test2

In [ ]:
#cd working

In [ ]:
#cd Train_Valid_images

In [ ]:
#!rm -rf Train_Valid_images 

In [ ]:
!cp -r ../input/rsna-jpeg-stage-2/Train_Valid_images/Train_Valid_images/ "Train_Valid_images"

In [ ]:
!cp -r ../input/rsna-jpeg-stage-2/Test_images/Test_images/ "Test_images"

Converting train images into jpg

In [ ]:

for filename in sorted(tqdm(os.listdir("../input/rsna-pneumonia-detection-challenge/stage_2_train_images"))):
    array = pydicom.dcmread(os.path.join("../input/rsna-pneumonia-detection-challenge/stage_2_train_images",filename)).pixel_array
    image = np.repeat(array[:,:,np.newaxis],3,2)
#     print(array[0][0])
#     print(filename[:len(filename)-4])
    cv2.imwrite(os.path.join("Train_Valid_images",filename[:len(filename)-4]+".jpg"),image)
#     break;
  

In [ ]:
for filename in sorted(tqdm(os.listdir("../input/rsna-pneumonia-detection-challenge/stage_2_train_images"))):
    pass

In [ ]:
for filename in sorted(tqdm(os.listdir("/kaggle/working/Train_Valid_images"))):
    pass

Converting test images into jpg

In [ ]:
for filename in sorted(tqdm(os.listdir("../input/rsna-pneumonia-detection-challenge/stage_2_test_images"))):
    array = pydicom.dcmread(os.path.join("../input/rsna-pneumonia-detection-challenge/stage_2_test_images",filename)).pixel_array
    image = np.repeat(array[:,:,np.newaxis],3,2)
#     print(array[0][0])
#     print(filename[:len(filename)-4])
    cv2.imwrite(os.path.join("Test_images",filename[:len(filename)-4]+".jpg"),image)
#     break;
    

For sample test

In [ ]:
i = 1
for filename in sorted(tqdm(os.listdir("../input/rsna-pneumonia-detection-challenge/stage_2_test_images"))):
    array = pydicom.dcmread(os.path.join("../input/rsna-pneumonia-detection-challenge/stage_2_test_images",filename)).pixel_array
    image = np.repeat(array[:,:,np.newaxis],3,2)
#     print(array[0][0])
#     print(filename[:len(filename)-4])
    cv2.imwrite(os.path.join("Test2",filename[:len(filename)-4]+".jpg"),image)
    i = i + 1
    if i > 100:
        break
#     break;

for male female 

train data

In [ ]:
for paths in raw_train_df['path']:
    array = pydicom.dcmread(paths).pixel_array
    image = np.repeat(array[:,:,np.newaxis],3,2)
#     print(array[0][0])
    filename = os.path.basename(paths)
     #print(filename[:len(filename)-4])
    cv2.imwrite(os.path.join("Train_Valid_images",filename[:len(filename)-4]+".jpg"),image)
#     break;

test data

In [ ]:
for paths in remain_df['path']:
    array = pydicom.dcmread(paths).pixel_array
    image = np.repeat(array[:,:,np.newaxis],3,2)
#     print(array[0][0])
    filename = os.path.basename(paths)
     #print(filename[:len(filename)-4])
    cv2.imwrite(os.path.join("Test_images",filename[:len(filename)-4]+".jpg"),image)
#     break;

import yaml
import json

Pneumonia = {
     "train" : "pneumonia_data/images/train",
    "val" : "pneumonia_data/images/validation",
    "nc" :  1,
    "names" : ["pneumonia"]
}

with open('Pneumonia.yaml', 'w') as outfile:
    yaml.safe_dump( Pneumonia, outfile, allow_unicode=True, default_flow_style=False)
#with open('file.yaml', 'w') as file:
    #file.write(json.dumps(Pneumonia)) 
 #    print(Pneumonia, file=file)
        


import yaml

yolov5sp = {
    # parameters
"nc" : 1,  # number of classes
"depth_multiple" : 0.33 , # model depth multiple
"width_multiple" : 0.50 , # layer channel multiple

# anchors
"anchors" :
  [[10,13, 16,30, 33,23] , # P3/8
   [30,61, 62,45, 59,119] , # P4/16
   [116,90, 156,198, 373,326]] , # P5/32

# YOLOv5 backbone
"backbone" :
  # [from, number, module, args]
  [[-1, 1, "Focus", [64, 3]],  # 0-P1/2
   [-1, 1, "Conv", [128, 3, 2]],  # 1-P2/4
   [-1, 3, "BottleneckCSP", [128]],
   [-1, 1, "Conv", [256, 3, 2]],  # 3-P3/8
   [-1, 9, "BottleneckCSP", [256]],
   [-1, 1, "Conv", [512, 3, 2]],  # 5-P4/16
   [-1, 9, "BottleneckCSP", [512]],
   [-1, 1, "Conv", [1024, 3, 2]],  # 7-P5/32
   [-1, 1, "SPP", [1024, [5, 9, 13]]],
   [-1, 3, "BottleneckCSP", [1024, False]],  # 9
  ],
# YOLOv5 head
"head" : 
  [[-1, 1, "Conv", [512, 1, 1]],
   [-1, 1, "nn.Upsample", [None, 2, 'nearest']],
   [[-1, 6], 1, "Concat", [1]],  # cat backbone P4
   [-1, 3, "BottleneckCSP", [512, False]],  # 13

   [-1, 1, "Conv", [256, 1, 1]],
   [-1, 1, "nn.Upsample", [None, 2, 'nearest']],
   [[-1, 4], 1, "Concat", [1]],  # cat backbone P3
   [-1, 3, "BottleneckCSP", [256, False]],  # 17 (P3/8-small)

   [-1, 1, "Conv", [256, 3, 2]],
   [[-1, 14], 1, "Concat", [1]],  # cat head P4
   [-1, 3, "BottleneckCSP", [512, False]],  # 20 (P4/16-medium)

   [-1, 1, "Conv", [512, 3, 2]],
   [[-1, 10], 1, "Concat", [1]],  # cat head P5
   [-1, 3, "BottleneckCSP", [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, "Detect", ["nc", "anchors"]],  # Detect(P3, P4, P5)
  ]
}

with open('yolov5sp.yaml', 'w') as outfile:
    yaml.safe_dump( yolov5sp, outfile, indent=4,  allow_unicode=False, default_flow_style=False)

In [ ]:
pwd

In [ ]:
import yaml

vgg16 = {
    # parameters
"nc" : 1,  # number of classes
"depth_multiple" : 1.0 , # model depth multiple
"width_multiple" : 1.0 , # layer channel multiple

# anchors
"anchors" :
  [[10,13, 16,30, 33,23] , # P3/8
   [30,61, 62,45, 59,119] , # P4/16
   [116,90, 156,198, 373,326]] , # P5/32

# YOLOv5 backbone
"backbone" :
  # [from, number, module, args]
  [[-1, 1, "Conv", [64, 3, 1]],  # 0
   [-1, 1, "Conv", [64, 3, 1]],
   [-1, 1, "Maxpool", [2, 2]],   # 2-P1/2
   [-1, 1, "Conv", [128, 3, 1]],
   [-1, 1, "Conv", [128, 3, 1]],
   [-1, 1, "Maxpool", [2, 2]],   # 5-P2/4
   [-1, 1, "Conv", [256, 3, 1]],
   [-1, 1, "Conv", [256, 3, 1]],
   [-1, 1, "Maxpool", [2, 2]],   # 8-P3/8
   [-1, 1, "Conv", [512, 3, 1]],
   [-1, 1, "Conv", [512, 3, 1]],
   [-1, 1, "Conv", [512, 3, 1]],
   [-1, 1, "Maxpool", [2, 2]],   # 13-P4/16
   [-1, 1, "Conv", [512, 3, 1]],
   [-1, 1, "Conv", [512, 3, 1]],
   [-1, 1, "Conv", [512, 3, 1]],
   [-1, 1, "Maxpool", [2, 2]],   # 16-P5/32
  ],

# YOLOv5 head
"head" : 
     [[-1, 1, "Bottleneck", [1024, False]],
   [-1, 2, "Bottleneck", [1024, False]],  # 18

   [-1, 1, "Conv", [256, 1, 1]],
   [-1, 1, "nn.Upsample", [None, 2, "nearest"]],
   [[-1, 14], 1, "Concat", [1]],  # concat backbone P4
   [-1, 1, "Bottleneck", [512, False]],
   [-1, 2, "Bottleneck", [512, False]],  # 23

   [-1, 1, "Conv", [128, 1, 1]],
   [-1, 1, "nn.Upsample", [None, 2, "nearest"]],
   [[-1, 9], 1, "Concat", [1]],  # concat backbone P3
   [-1, 1, "Bottleneck", [256, False]],
   [-1, 2, "Bottleneck", [256, False]],  # 28

   [[28, 23, 18], 1, "Detect", ["nc", "anchors"]],   # Detect(P3, P4, P5)
  ]
}

with open('vgg16_1.yaml', 'w') as outfile:
    yaml.safe_dump(vgg16, outfile, indent=4,  allow_unicode=False, default_flow_style=False)

In [ ]:
# parameters
nc: 1  # number of classes
depth_multiple: 1.0  # model depth multiple
width_multiple: 1.0  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# VGG16 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 3, 1]],  # 0
   [-1, 1, Conv, [64, 3, 1]],
   [-1, 1, Maxpool, [2, 2]],   # 2-P1/2
   [-1, 1, Conv, [128, 3, 1]],
   [-1, 1, Conv, [128, 3, 1]],
   [-1, 1, Maxpool, [2, 2]],   # 5-P2/4
   [-1, 1, Conv, [256, 3, 1]],
   [-1, 1, Conv, [256, 3, 1]],
   [-1, 1, Maxpool, [2, 2]],   # 8-P3/8
   [-1, 1, Conv, [512, 3, 1]],
   [-1, 1, Conv, [512, 3, 1]],
   [-1, 1, Conv, [512, 3, 1]],
   [-1, 1, Maxpool, [2, 2]],   # 13-P4/16
   [-1, 1, Conv, [512, 3, 1]],
   [-1, 1, Conv, [512, 3, 1]],
   [-1, 1, Conv, [512, 3, 1]],
   [-1, 1, Maxpool, [2, 2]],   # 16-P5/32
  ]

# YOLOv3 head
head:
  [[-1, 1, Bottleneck, [1024, False]],
   [-1, 2, Bottleneck, [1024, False]],  # 18

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, "nearest"]],
   [[-1, 14], 1, Concat, [1]],  # concat backbone P4
   [-1, 1, Bottleneck, [512, False]],
   [-1, 2, Bottleneck, [512, False]],  # 23

   [-1, 1, Conv, [128, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, "nearest"]],
   [[-1, 9], 1, Concat, [1]],  # concat backbone P3
   [-1, 1, Bottleneck, [256, False]],
   [-1, 2, Bottleneck, [256, False]],  # 28

   [[28, 23, 18], 1, Detect, [number_classes, anchors]],   # Detect(P3, P4, P5)
  ]

In [ ]:
cd "/kaggle/working"

In [ ]:
! git clone https://github.com/ultralytics/yolov5.git

In [ ]:
cp -r ../input/yolov5-mod/yolov5-master .

In [ ]:
!cp "../input/yolov5-pneumonia-yaml/Pneumonia.yaml" "yolov5-master/"
!cp "../input/flexible-yolov5-efficient/vgg16.yaml" "yolov5-master/models/"  #../input/yolov5-pneumonia-yaml/yolov5sp.yaml
os.chdir("yolov5-master")
!rm -rf pneumonia_data
!mkdir pneumonia_data
os.chdir("pneumonia_data")
!mkdir images
!mkdir labels
os.chdir("images")
!mkdir train
!mkdir validation
!mkdir test
os.chdir(os.pardir)
os.chdir("labels")
!mkdir train
!mkdir validation
!mkdir test
os.chdir(os.pardir)
os.chdir(os.pardir)

In [ ]:
!tree

In [ ]:
import pandas as pd
import os
import ast
import shutil
from sklearn import model_selection
from tqdm import tqdm
import numpy as np

In [ ]:

!pip install -r requirements.txt

In [ ]:
pip install -U PyYAML

In [ ]:
DATA_PATH = "../input/rsna-pneumonia-detection-challenge/"
OUTPUT_PATH = "/kaggle/working/yolov5-master/pneumonia_data"

In [ ]:
def process_data(data,data_type="train"):
    for _,row in tqdm(data.iterrows(),total=len(data),position=0,leave=True):
        image_name = row["patientId"]
        bounding_boxes = row["bboxes"]
        yolo_data = []
        for bbox in bounding_boxes:
            if(bbox[4]=="0.0"):
              # print("hello")
              continue
            x=float(bbox[0])
            y=float(bbox[1])
            w=float(bbox[2])
            h=float(bbox[3])
            x_center = x+w/2
            y_center = y+h/2
            x_center/=1024.0
            y_center/=1024.0
            w/=1024.0
            h/=1024.0
            yolo_data.append([0,x_center,y_center,w,h])
        yolo_data = np.array(yolo_data)
        np.savetxt(
            os.path.join(OUTPUT_PATH,f"labels/{data_type}/{image_name}.txt"),
            yolo_data,
            fmt = ["%d","%f","%f","%f","%f"]
        )
        shutil.copyfile(
            f"/kaggle/working/Train_Valid_images/Train_Valid_images/{image_name}.jpg",
            os.path.join(OUTPUT_PATH,f"images/{data_type}/{image_name}.jpg")
        )

In [ ]:
def convert_from_txt(file):
    prediction_string= ""
    if(os.path.isfile('runs/detect/exp/labels/'+file+'.txt')):
        words = convert_from_yolo_to_bbox('runs/detect/exp/labels/'+file+'.txt')
        #return words
        for i in range(len(words)):

            clss = float(words[i][0])
            x_center = float(words[i][1])
            y_center = float(words[i][2])
            w = float(words[i][3])
            h = float(words[i][4])
            conf = float(words[i][5])
            w = w * 1024
            h = h * 1024
            x_center = x_center * 1024
            y_center = y_center * 1024
            x = x_center - w/2
            y = y_center - h/2
            if (prediction_string != ''):
                prediction_string = prediction_string + ' '
            prediction_string = prediction_string + str(round(conf,3))+ ' '+ str(int(x)) + ' ' + str(int(y)) + ' '+ str(int(w)) + ' '+ str(int(h))
            
      
        
    return prediction_string

str(int((conf * 100) + 0.5) / 100.0)

str(round(conf,3))

In [ ]:
def convert_from_yolo_to_bbox(txt_file):
    
    f = open(txt_file, "r")
    lines = f.readlines()
    words = []
    for line in lines:
        word = line.split(" ") 
        words.append(word)
    return words

In [ ]:
#ss = convert_from_yolo_to_bbox("runs/detect/exp3/labels/00991acc-85b3-41c7-a397-bdf925c3697a.txt")

In [ ]:
ss = convert_from_txt("001031d9-f904-4a23-b3e5-2c088acd19c6")

In [ ]:
ss

In [ ]:
len(ss)

In [ ]:
ss

In [ ]:
float(ss[0][4])

In [ ]:
os.chdir(os.pardir)

In [ ]:
os.getcwd()

In [ ]:
cd ..

In [ ]:
#!rm -rf yolov5-master

In [ ]:
#!pip uninstall -q -y wandb

In [ ]:
DATA_PATH

In [ ]:
pwd

in kaggle/working dir

In [ ]:
if __name__=="__main__":
    full_path = str(os.path.join(DATA_PATH,"stage_2_train_labels.csv"))
    print(full_path)
    df = pd.read_csv(full_path)
   # df_test = df_full.iloc[int(0.9*len(df_full)):]
    #df = raw_train_df.reset_index(drop=True) #male female
    df = df[df['Target']==1]
    # df.bbox = df.bbox.apply(ast.literal_eval)
    cols = ['x', 'y', 'width','height','Target']
    df['combined'] = df[cols].apply(lambda row: ','.join(row.values.astype(str)), axis=1)
    df['combined'] = df.combined.apply(lambda x: x.strip('()').split(','))
    # print(type(df['combined'].iloc[0]))
    # print(df)
    df = df.groupby("patientId")['combined'].apply(list).reset_index(name="bboxes")
    # print(df)
    df_train,df_valid = model_selection.train_test_split(
        df,
        test_size=0.11,
        random_state=True,
        shuffle=True,
    )
    df_train = df_train.reset_index(drop=True)
    df_valid = df_valid.reset_index(drop=True)
    # print(df_train)
    process_data(df_train,data_type="train")
    process_data(df_valid,data_type="validation")

In [ ]:
pwd

In [ ]:
raw_train_df.head()

In [ ]:
raw_train_df.shape

In [ ]:
full_path = str("../input/rsna-pneumonia-detection-challenge/stage_2_train_labels.csv")
print(full_path)
df = pd.read_csv(full_path)
df.head()

In [ ]:
os.chdir("yolov5-master")
os.getcwd()

may be yolov5 yaml model architecture will be same as pretrained weights

In [ ]:
#!tree #see structure of directories

In [ ]:
#!python3 train.py --resume "../../input/yolov5-rsna/last.pt" 

In [ ]:
!python3 train.py --img 1024 --batch 8 --epochs 20 --data Pneumonia.yaml --cfg models/yolov5sp.yaml --name wm --weights "https://github.com/ultralytics/yolov5/releases/download/v5.0/yolov5l6.pt" --project yolov5-rsna   --single-cls #https://github.com/ultralytics/yolov5/releases/download/v5.0/yolov5l6.pt  ##"../../input/d/sakib01/yolov5-rsna/last.pt"

train with vgg16

In [ ]:
!wget 'https://download.pytorch.org/models/vgg16-397923af.pth'

In [ ]:
 os.rename('vgg16-397923af.pth', "vgg16-397923af"+".pt")

In [ ]:
!python3 train.py --img 1024 --batch 8 --epochs 2 --data Pneumonia.yaml --cfg models/vgg16.yaml --name wm_vgg16_test_1 --weights "vgg16-397923af.pt" --project yolov5-rsna    --single-cls  #https://github.com/ultralytics/yolov5/releases/download/v5.0/yolov5l6.pt  ##"../../input/d/sakib01/yolov5-rsna/last.pt"

In [ ]:
!python3 train.py --img 1024 --batch 8 --epochs 10 --data Pneumonia.yaml --cfg models/yolov5sp.yaml --name wm --weights "yolov5-rsna/wm/weights/last.pt" --project yolov5-rsna   --single-cls #https://github.com/ultralytics/yolov5/releases/download/v5.0/yolov5l6.pt  ##"../../input/d/sakib01/yolov5-rsna/last.pt"

In [ ]:
!python detect.py --source "/kaggle/working/Test2/"  --weights "../../input/d/sakib01/yolov5-rsna/last.pt" --save-txt --conf-thres  0.3 --iou-thres 0.5 #--iou-thres 0.3 --conf-thres 0.6/ --conf 0

In [ ]:
!python detect.py --source "/kaggle/working/Test2/"  --weights "../../input/d/sakib01/yolov5-rsna/last.pt" --save-txt --save-conf --conf-thres  0.3 --iou-thres 0.5 #--iou-thres 0.3 --conf-thres 0.6/ --conf 0

In [ ]:
cd working

In [ ]:
cd ./flexible-yolov5-main

for flexible yolov5

In [ ]:
import sys

sys.path.append('.')
from od.models.modules.experimental import *
from od.data.datasets import letterbox
from utils.general import *
from utils.split_detector import SPLITINFERENCE
from utils.torch_utils import *


class Detector(object):
    def __init__(self, pt_path, img_size, conf_thres=0.4, iou_thres=0.3, classes=0, agnostic_nms=False,
                 xcycwh=True, device=0):
        self.pt_path = pt_path
        self.img_size = img_size
        self.device = torch.device('cuda:{}'.format(device))
        self.model = self.load_model()
        self.conf_thres = conf_thres
        self.iou_thres = iou_thres
        self.classes = classes
        self.agnostic_nms = agnostic_nms
        self.xcycwh = xcycwh
       # self.class_names = self.load_class_names(namesfile)

    def load_model(self):
        model = attempt_load(self.pt_path, map_location=self.device)  # load FP32 model
        return model

    

    def __call__(self, ori_img, split_width=1, split_height=1):
        if split_width == 1 and split_height == 1:
            bboxes, scores, ids = self.detect_image(ori_img)
        else:
            bboxes = []
            scores = []
            ids = []
            output = self.detect_img_split(image=ori_img, split_width=split_width, split_height=split_height)['data']
            for key in output.keys():
                values = output[key]
                for value in values:
                    x_min = value[0]
                    y_min = value[1]
                    x_max = value[2]
                    y_max = value[3]
                    w = x_max - x_min
                    h = y_max - y_min
                    if self.xcycwh:
                        bboxes.append([x_min + w / 2, y_min + h / 2, w, h])
                    else:
                        bboxes.append(value[:4])
                    scores.append(value[4])
                    ids.append(key)
        return np.asarray(bboxes), np.asarray(scores), np.asarray(ids)

    def detect_image(self, image):
        bboxes = []
        scores = []
        ids = []
        im0s = image
        img = letterbox(im0s, new_shape=self.img_size)[0]
        img = img[:, :, ::-1].transpose(2, 0, 1)
        img = np.ascontiguousarray(img)
        img = torch.from_numpy(img).to(self.device)
        img = img.float()
        img /= 255.0
        if img.ndimension() == 3:
            img = img.unsqueeze(0)
        pred = self.model(img)[0]
        pred = non_max_suppression(pred, self.conf_thres, self.iou_thres, classes=self.classes,
                                   agnostic=self.agnostic_nms)
        for i, det in enumerate(pred):
            if det is not None and len(det):
                det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0s.shape).round()
                for *xyxy, conf, cls in det:
                    x_min = xyxy[0].cpu()
                    y_min = xyxy[1].cpu()
                    x_max = xyxy[2].cpu()
                    y_max = xyxy[3].cpu()
                    score = conf.cpu()
                    clas = cls.cpu()
                    w = x_max - x_min
                    h = y_max - y_min
                    if self.xcycwh:
                        # center coord, w, h
                        bboxes.append([x_min + w / 2, y_min + h / 2, w, h])
                    else:
                        bboxes.append([x_min, y_min, x_max, y_max])
                    scores.append(score)
                    ids.append(clas)
        return np.asarray(bboxes), np.asarray(scores), np.asarray(ids)

    @SPLITINFERENCE(split_width=2, split_height=1)
    def detect_img_split(self, image='', **kwargs):
        outputs_json = {}
        im0s = image
        img = letterbox(im0s, new_shape=self.img_size)[0]
        img = img[:, :, ::-1].transpose(2, 0, 1)
        img = np.ascontiguousarray(img)
        img = torch.from_numpy(img).to(self.device)
        img = img.float()
        img /= 255.0
        if img.ndimension() == 3:
            img = img.unsqueeze(0)
        pred = self.model(img)[0]
        pred = non_max_suppression(pred, self.conf_thres, self.iou_thres, classes=self.classes,
                                   agnostic=self.agnostic_nms)
        for i, det in enumerate(pred):
            if det is not None and len(det):
                det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0s.shape).round()
                for *xyxy, conf, cls in det:
                    x_min = xyxy[0].cpu()
                    y_min = xyxy[1].cpu()
                    x_max = xyxy[2].cpu()
                    y_max = xyxy[3].cpu()
                    score = conf.cpu()
                    clas = cls.cpu()
                    if clas in outputs_json:
                        outputs_json[clas].append([x_min, y_min, x_max, y_max, score])
                    else:
                        outputs_json[clas] = [[x_min, y_min, x_max, y_max, score]]
        return {'data': outputs_json}

In [ ]:
pt_path = '../../input/yolov5-efficientnetb7/best.pt'
model = Detector(pt_path,  512, xcycwh=False)
imgs_root = "/kaggle/working/Test_images/Test_images/"
imgs = os.listdir(imgs_root)
save_dir = '/kaggle/working/output/'
if not os.path.exists(save_dir):
    os.mkdir(save_dir)
for img in imgs:
    im = cv2.imread(os.path.join(imgs_root, img))
 
    bboxes, scores, ids = model.detect_image(im)
 
    for idx in range(bboxes.shape[0]):
        bbox = bboxes[idx].astype(int)
        score = scores[idx]
        cv2.rectangle(im, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (0, 0, 255), 2, 1)
    cv2.imwrite(os.path.join(save_dir, img), im)

In [ ]:
!python detect.py --source "/kaggle/working/Test_images/"  --weights "../../input/yolov5-efficientnetb7/best.pt" --save-txt --save-conf --conf-thres  0.3 --iou-thres 0.5 #--iou-thres 0.3 --conf-thres 0.6/ --conf 0

In [ ]:
sample_df = pd.read_csv('../../input/rsna-pneumonia-detection-challenge/stage_2_sample_submission.csv')

In [ ]:
sample_df.head()

In [ ]:
test_df = sample_df

In [ ]:
for i in range(test_df.shape[0]):
    test_df['PredictionString'][i] = convert_from_txt(test_df["patientId"][i])
    

In [ ]:
test_df

In [ ]:
!dir

In [ ]:
cd ..

In [ ]:
test_df.to_csv('test_submission_final.csv')

In [ ]:
import tarfile
import os.path

def make_tarfile(output_filename, source_dir):
    with tarfile.open(output_filename, "w:gz") as tar:
        tar.add(source_dir, arcname=os.path.basename(source_dir))

In [ ]:
cd working

In [ ]:
make_tarfile("output_yolov5_efficient.tar",'/kaggle/working/output/')


In [ ]:
make_tarfile("weights_25_epochs_25_11_12.tar",'/kaggle/working/yolov5/runs')

In [ ]:
cd 'yolov5'

In [ ]:
cd 'Train_Valid_images'

In [ ]:
!dir

In [ ]:
pwd

In [ ]:
cd ..

In [ ]:
!git clone https://github.com/yl305237731/flexible-yolov5.git

In [ ]:
!rm -rf flexible-yolov5-main

In [ ]:
cp -r ../input/flexible-yolov5/flexible-yolov5-main .

In [ ]:
!pip install -r flexible-yolov5-main/requirements.txt

In [ ]:
!cp "../input/yolov5-pneumonia-yaml/Pneumonia.yaml" "flexible-yolov5-main"

In [ ]:
!cp "../input/flexible-yolov5-efficient/Flexible_Yolo_efficientnet.yaml" "flexible-yolov5-main"

In [ ]:
cd ..

kaggle/working

In [ ]:
def process_data(data,data_type="train"):
    for _,row in tqdm(data.iterrows(),total=len(data),position=0,leave=True):
        image_name = row["patientId"]
        bounding_boxes = row["bboxes"]
        yolo_data = []
        if bounding_boxes != 0:          
            for bbox in bounding_boxes:
                if(bbox[4]=="0.0"):
                  # print("hello")
                  continue
                x=float(bbox[0])
                y=float(bbox[1])
                w=float(bbox[2])
                h=float(bbox[3])
                x_center = x+w/2
                y_center = y+h/2
                x_center/=1024.0
                y_center/=1024.0
                w/=1024.0
                h/=1024.0
                yolo_data.append([0,x_center,y_center,w,h])
            yolo_data = np.array(yolo_data)
            np.savetxt(
                os.path.join(OUTPUT_PATH,f"labels/{data_type}/{image_name}.txt"),
                yolo_data,
                fmt = ["%d","%f","%f","%f","%f"]
            )
            
        shutil.copyfile(
            f"../input/rsna-jpeg-stage-2/Train_Valid_images/Train_Valid_images/{image_name}.jpg",
            os.path.join(OUTPUT_PATH,f"images/{data_type}/{image_name}.jpg")
        )

In [ ]:
if __name__=="__main__":
    full_path = str(os.path.join(DATA_PATH,"stage_2_train_labels.csv"))
    print(full_path)
    df = pd.read_csv(full_path)
    df1 = df[df['Target']==1]
    # df.bbox = df.bbox.apply(ast.literal_eval)
    cols = ['x', 'y', 'width','height','Target']
    df1['combined'] = df1[cols].apply(lambda row: ','.join(row.values.astype(str)), axis=1)
    df1['combined'] = df1.combined.apply(lambda x: x.strip('()').split(','))
    df1 = df1.groupby("patientId")['combined'].apply(list).reset_index(name="bboxes")

    df2 = df[df['Target']!=1]
    df2.drop(cols, axis  = 1, inplace = True)
    df2['bboxes'] = 0
    df = pd.concat([df1,df2])
    df = df.sample(df.shape[0]).reset_index()
    # print(df)
    df_train,df_valid = model_selection.train_test_split(
        df,
        test_size=0.11,
        random_state=True,
        shuffle=True,
    )
    df_train = df_train.reset_index(drop=True)
    df_valid = df_valid.reset_index(drop=True)
    # print(df_train)
    process_data(df_train,data_type="train")
    process_data(df_valid,data_type="validation")

In [ ]:
os.chdir("flexible-yolov5-main")

In [ ]:

!rm -rf pneumonia_data
!mkdir pneumonia_data
os.chdir("pneumonia_data")
!mkdir images
!mkdir labels
os.chdir("images")
!mkdir train
!mkdir validation
!mkdir test
os.chdir(os.pardir)
os.chdir("labels")
!mkdir train
!mkdir validation
!mkdir test
os.chdir(os.pardir)
os.chdir(os.pardir)

In [ ]:
DATA_PATH = "../input/rsna-pneumonia-detection-challenge/"
OUTPUT_PATH = "/kaggle/working/flexible-yolov5-main/pneumonia_data"

In [ ]:
%cp ../../input/flexible-yolov5-efficient/model_resnet.yaml .

In [ ]:
!python3 scripts/train.py --img 512 --batch 8 --epochs 15 --data Pneumonia.yaml --cfg configs/model_efficientnet.yaml --name wm2_efficientnetb7_pretrained_with_negative_images --project yolov5-rsna   --single-cls #https://github.com/ultralytics/yolov5/releases/download/v5.0/yolov5l6.pt  ##"../../input/d/sakib01/yolov5-rsna/last.pt"

In [ ]:
!wget 'https://download.pytorch.org/models/resnet50-19c8e357.pth'

In [ ]:
 os.rename('resnet50-19c8e357.pth', "resnet50-19c8e357"+".pt")

In [ ]:
pwd

In [ ]:
!python3 scripts/train.py --img 1024 --batch 8 --epochs 10 --data Pneumonia.yaml --cfg model_resnet.yaml  --name wm2_resnet50_flexible_yolov5 --project yolov5-rsna   --single-cls #https://github.com/ultralytics/yolov5/releases/download/v5.0/yolov5l6.pt  ##"../../input/d/sakib01/yolov5-rsna/last.pt"

In [ ]:
!python3 scripts/train.py --img 1024 --batch 8 --epochs 10 --data Pneumonia.yaml --cfg configs/model_densenet.yaml   --name wm2_densenet121_flexible_yolov5 --project yolov5-rsna   --single-cls #https://github.com/ultralytics/yolov5/releases/download/v5.0/yolov5l6.pt  ##"../../input/d/sakib01/yolov5-rsna/last.pt"

In [ ]:
cd ..

In [ ]:
make_tarfile("efficientnetb7_weights_15_epochs_14_12_21.tar",'/kaggle/working/flexible-yolov5-main/yolov5-rsna/wm2_efficientnet_pretrained2/weights')